In [ ]:
import pandas as pd
import random as rnd
import networkx as nx
from networkx.algorithms import bipartite #we load the bipartite algorithms to facilitate writing the code
import numpy as np
import matplotlib.pyplot as plt
from Functions import *

# Checking for statistical relevance

## Hypothesis Testing

A commonplace task in statistical inferences
is calculating the probability of observing a value or something more extreme
under an assumed "null" model of reality.
This is what we commonly call "hypothesis testing",
and where the oft-misunderstood term "p-value" shows up.
### Hypothesis testing in coin flips, by simulation

As an example, hypothesis testing in coin flips follows this logic:

- I observe that 8 out of 10 coin tosses give me heads, giving me a probability of heads $p=0.8$ (a summary statistic).
- Under a "null distribution" of a fair coin, I simulate the distribution of probability of heads (the summary statistic) that I would get from 10 coin tosses.
- Finally, I use that distribution to calculate the probability of observing $p=0.8$ or more extreme.

### Hypothesis testing in graphs

The same protocol applies when we perform hypothesis testing on graphs.

Firstly, we calculate a _summary statistic_ that describes our graph.

Secondly, we propose a _null graph model_, and calculate our summary statistic under simulated versions of that null graph model.

Thirdly, we look at the probability of observing the summary statistic value that we calculated in step 1 or more extreme, under the assumed graph null model distribution.

## Null models ensembles

### Z-Scores and P-values

To really asses how different from the random expectation of a null model a given empirical netowrk is, we need to compare out empirical values with the distribution of values in the **random ensemble**.

A random ensemble is composed by **$N_{rep}$** random replicates of an empirical network. Dpending on the null model that we use to make the randomizations we will talk about the **ER ensemble**, or **configuration model ensemble**. 

As we saw before, by comparing our network to the average and the standard deviation of the values in the random ensemble we can determine how statistically significant is the empirical value of the structural feature we are studying.

Let's see the significan. 
To create a random ensemble we will specify the number of random networks to sample,a nd decide what models to use. We can use more than one, and see how incrementing the model complexity helps us to understand what can be contributing to the empirical structure

In [ ]:
## read network
G = nx.karate_club_graph()

In [ ]:
#prepare storage
#determine number of samplings from the random ensemble
Nrep = 300  # Replace this with your desired number of repetitions
repetitions = range(Nrep + 1)  # Repetition numbers from 0 to Nrep
ensembles = ["ER", "CONF"]  # Ensemble names
metrics = ["Q", "C"]  # Metric names to asses

# Create the MultiIndex
multi_index = pd.MultiIndex.from_product([repetitions, ensembles], names=["Repetition", "Ensemble"])

# Create an empty DataFrame with this MultiIndex
Value_df = pd.DataFrame(index=multi_index, columns=metrics)   

In [ ]:
Value_df.head()

In [ ]:
#Let's start by measuring the metrics in the empirical network
#clustering
C_emp=nx.average_clustering(G)
#modularity: get partition
partition = nx.community.louvain_communities(G)
# Calculate the modularity
Q_emp=nx.community.quality.modularity(G, partition)
empirical_values = pd.Series({'Q': Q_emp, 'C': C_emp})
print(empirical_values)

In [ ]:
#prepare parameters for null models
N=G.number_of_nodes()
L=G.number_of_edges()
K=pd.Series(dict(G.degree))

In [ ]:
#Noew let's fill the dataframe
for rep in range(Nrep+1):
    #generate the randomization in first null model
    G_ER=nx.gnm_random_graph(N, L)
    
    #measure quantites:
    C=nx.average_clustering(G_ER)
    
    #modularity: get partition
    partition = nx.community.louvain_communities(G_ER)
    # Calculate the modularity
    Q=nx.community.quality.modularity(G_ER, partition)
    
    Value_df.loc[(rep,"ER"),"C"]=C
    Value_df.loc[(rep,"ER"),"Q"]=Q
    
    #generate the randomization in first null model
    G_CF=nx.configuration_model(K, create_using=nx.Graph())
    
    #measure quantites:
    C=nx.average_clustering(G_CF)
    
    #modularity: get partition
    partition = nx.community.louvain_communities(G_CF)
    # Calculate the modularity
    Q=nx.community.quality.modularity(G_CF, partition)
    
    Value_df.loc[(rep,"CONF"),"C"]=C
    Value_df.loc[(rep,"CONF"),"Q"]=Q  
    
    
Value_df.head()   

Now we want to analize how different from the ranfom expectations is my network. For that let's plot the distribution of values in the random ensembles, and compare where lies the empirical value.

In [ ]:
# Filter the data to only include the ER model -
df_er = Value_df.xs('ER', level='Ensemble')
# Filter the data to only include the CONF model -
df_conf = Value_df.xs('CONF', level='Ensemble')

# Plotting the distributions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5)) #one plot for each quantity

# Plotting Q distribution
ax1.hist(df_er['Q'], bins=10, color='lightgreen', edgecolor='black', alpha=0.6)
ax1.hist(df_conf['Q'], bins=10, color='orange', edgecolor='black', alpha=0.6)
ax1.axvline(Q_emp, color='k', linestyle='--')
ax1.set_title('Distribution of Q')
ax1.set_xlabel('Q values')
ax1.set_ylabel('Frequency')

# Plotting C distribution
ax2.hist(df_er['C'], bins=10, color='lightgreen', edgecolor='black',alpha=0.6)
ax2.hist(df_conf['C'], bins=10, color='orange', edgecolor='black',alpha=0.6)
ax2.axvline(C_emp, color='k', linestyle='--')
ax2.set_title('Distribution of C')
ax2.set_xlabel('C values')
ax2.set_ylabel('Frequency')

# Display the plots
plt.tight_layout()
plt.show()

In [ ]:
# Calculate mean and std for each metric and ensemble
model_stats = Value_df.groupby('Ensemble').agg(['mean', 'std'])

# Initialize z_scores DataFrame
z_scores_df = pd.DataFrame(index=['ER', 'CONF'], columns=['Q', 'C'])

# Compute z-scores
for metric in ['Q', 'C']:
    for ensemble in ['ER', 'CONF']:
        mean_value = model_stats.loc[ensemble, (metric, 'mean')]
        std_value = model_stats.loc[ensemble, (metric, 'std')]
        empirical_value = empirical_values[metric]
        
        # Calculate z-score
        z_score = (empirical_value - mean_value) / std_value if std_value != 0 else None
        z_scores_df.loc[ensemble, metric] = z_score


In [ ]:
print(z_scores_df)

<div class="alert alert-block alert-success"><b>Up to you: </b>
<h4> Exercise 22</h4>
    
- 1. compare the different motifs composition in the ER ensemble and the ensemble that keeps the degree sequence of the St. Marks estuary food-web. 
    Do only 5 repetitions each network (**$N_{rep}=4$**)
    

</div>

In [ ]:
#read the edgelist and create the network
filename="./data/WoL_StMarks/st_marks_Ilist.csv"
Ilist=pd.read_csv(filename, header=None, index_col=None)
Ilist.columns=["source","target","w"]
FW=nx.from_pandas_edgelist(Ilist, edge_attr="w", create_using=nx.DiGraph)
#your code here below
# 1. Prepare the number of repetitions and the indexes for the dataframe
# 2. Creat the empty dataframe
# 3. prepare the parameters for the null models
# 4. cuantify the motifs in the empirical network and save them in a series
# 5. For each repetition, store the series of motifs indexed by repetition and model type
# 6. Once the dataframe is filled, create a figure with two images (one for each model) and plot for each motif the values in the random ensemble
# 7. OVerlay the value of the empirical ensemble

In [ ]:
# %load ./snippets/ex22a.py


In [ ]:
# %load ./snippets/ex22b.py


<div class="alert alert-block alert-success"><b>Up to you: </b>
<h4> Exercise 23</h4>
    
- 1. Compare the values of nestedness of the Doñana pollination network in the ER, CONF and KSEQ ensembles. 
    Do only 100 repetitions
    
</div>

In [ ]:
# %load ./snippets/ex23a.py


In [ ]:
# %load ./snippets/ex23b.py
